In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.8.0


In [18]:
x_data = tf.constant([[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]], dtype=tf.float32)
y_data = tf.constant([[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]], dtype=tf.float32)

In [19]:
nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [20]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-1.2838469 , -1.1120201 , -1.1682403 ],
       [-0.02682939, -1.2233621 ,  0.9378554 ],
       [-1.259596  , -2.262079  ,  0.72499883],
       [ 0.28997898, -0.27300495, -2.026413  ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.27028295, -0.45455965, -0.08970138], dtype=float32)>


In [21]:
def hypothesis(features):
    X = tf.matmul(features, W)
    # softmax -> tf.nn.softmax(X + b)
    # tf.reduce_sum(tf.exp(X+b), keepdims=True, axis=1) -> tf.exp(X+b)행렬의 가로축에 대해 합을 구한다 / keepdims=True -> 행렬의 차원을 유지한다
    result = tf.divide(tf.exp(X+b), tf.reduce_sum(tf.exp(X+b), keepdims=True, axis=1))
    
    return result

In [22]:
# cross-entropy
def cost_fn(x, y):
    logits = hypothesis(x)
    cost = -tf.reduce_sum(y * tf.math.log(logits))
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

In [23]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
    
    grads = tape.gradient(loss, [W, b])
        
    return grads

In [24]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        optimizer.apply_gradients(zip(grad_fn(X, Y), [W, b]))
        if(i==0) | ((i+1)%verbose == 0):
             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
fit(x_data, y_data)

Loss at epoch 1: 51.559467
Loss at epoch 100: 80.505074
Loss at epoch 200: 80.402649
Loss at epoch 300: 56.206383
Loss at epoch 400: 70.501694
Loss at epoch 500: 7.710432
Loss at epoch 600: 12.445446
Loss at epoch 700: 39.128941
Loss at epoch 800: 56.990528
Loss at epoch 900: 45.193378
Loss at epoch 1000: 0.026957


In [25]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
# tf.argmax -> 배열에서 가장 큰 값을 가진 인덱스 값 반환
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[6.561945e-13 1.349348e-03 9.986507e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [26]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[2.39373876e-25 5.75330148e-12 1.00000000e+00]
 [6.56194500e-13 1.34934799e-03 9.98650730e-01]
 [4.02004181e-36 2.28441739e-03 9.97715592e-01]
 [6.74858040e-30 9.99827266e-01 1.72801578e-04]
 [1.06920451e-02 9.87427592e-01 1.88037404e-03]
 [5.64331344e-07 9.99999404e-01 3.38681825e-15]
 [9.89571393e-01 1.04286745e-02 2.39589707e-13]
 [9.99988914e-01 1.11672180e-05 2.67333691e-20]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
